In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
torch.manual_seed(42)

In [ ]:
data = pd.read_csv("fashion-mnist_train.csv")

In [ ]:
data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X=data.iloc[:,1:]
Y=data.iloc[:,0]

In [ ]:
X=X/255.0

In [ ]:
X

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.019608,0.0,0.0,...,0.000000,0.000000,0.000000,0.117647,0.168627,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.003922,0.007843,0.0,0.0,0.000000,0.0,0.0,...,0.011765,0.000000,0.000000,0.000000,0.000000,0.003922,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
59996,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.286275,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
59997,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.627451,0.635294,0.639216,0.529412,0.368627,0.000000,0.0,0.0,0.0,0.0
59998,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [ ]:
X_train_tensor= torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor= torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor= torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor= torch.tensor(y_test.values, dtype=torch.long)

In [ ]:
class CoustemDataset(Dataset):
  def __init__(self, features, labels):
    self.features=features
    self.labels=labels
  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self, index):
     return self.features[index], self.labels[index]

In [ ]:
train_dataset= CoustemDataset(X_train_tensor,y_train_tensor)
test_dataset= CoustemDataset(X_test_tensor,y_test_tensor)

In [ ]:
class MYNN(nn.Module):
  def __init__(self,input_dim, output_dim,num_hidden_layers,number_of_neurons_per_layer,dropout_rate,) -> None:
    super().__init__() #invoking the parent class's constructor

    layer=[]

    for i in range(num_hidden_layers):
      layer.append(nn.Linear(input_dim, number_of_neurons_per_layer))
      layer.append(nn.BatchNorm1d(number_of_neurons_per_layer))
      layer.append(nn.ReLU())
      layer.append(nn.Dropout(dropout_rate))
      input_dim=number_of_neurons_per_layer
    layer.append(nn.Linear(number_of_neurons_per_layer,output_dim))

    self.model=nn.Sequential(*layer) #it won't recive list of layers so we need to break them into induviuals parts using *


  def forward(self,X):
    return self.model(X)

In [ ]:
!pip install optuna

In [ ]:
import optuna

# def objective(trail):

#   #searc space
#   num_hidden_layers= trail.suggest_int("num_hidden_layers", 1, 5)
#   number_of_neurons_per_layer= trail.suggest_int("number_of_neurons", 32, 512, step=32)
#   learning_rate= trail.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
#   optimizer=trail.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
#   dropout_rate= trail.suggest_float("dropout_rate", 0.0, 0.5)
#   Batch_size=trail.suggest_categorical("Batch_size", [32,64,128,256,512])
#   weight_decay= trail.suggest_float("weight_decay", 1e-5, 1e-1, log=True)
#   epochs= trail.suggest_int("epochs", 10, 100, step=10)

#   #data loader

#   train_data_loader= DataLoader(train_dataset, batch_size=Batch_size, shuffle=True, pin_memory=True)
#   test_data_loader= DataLoader(test_dataset, batch_size=Batch_size, shuffle=False, pin_memory=True)

#   #model initialization
#   input_dim=X_train_tensor.shape[1]
#   output_dim=10
#   model= MYNN(input_dim,output_dim, num_hidden_layers,number_of_neurons_per_layer,dropout_rate)
#   model.to(device)
#   criterion= nn.CrossEntropyLoss()
#   if optimizer== "Adam":
#     optimizer= optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
#   elif optimizer=="SGD":
#     optimizer=optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
#   else:
#     optimizer=optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

#   #raining loop

#   for epoch in range(epochs):

#     for batch_features, batch_labels in train_data_loader:
#       batch_features, batch_labels= batch_features.to(device), batch_labels.to(device)
#       #all previous gradient into zero
#       optimizer.zero_grad()
#       #forward propagation
#       outputs= model(batch_features)
#       #calculate loss
#       loss= criterion(outputs, batch_labels)
#       #backward propagation
#       loss.backward()

#       #update parameters
#       optimizer.step()

#   #initialize model evaluaton
#   model.eval()

#   with torch.no_grad():
#     total=0
#     correct=0

#     for batch_features, batch_labels in test_data_loader:
#       batch_features, batch_labels= batch_features.to(device), batch_labels.to(device)
#       output= model(batch_features)

#       _, predicted= torch.max(output, 1)
#       total= total + batch_labels.shape[0]
#       correct= correct + (predicted==batch_labels).sum().item()

#     accuracy= correct/total
#     return accuracy


# objective function
def objective(trial):

  # next hyperparameter values from the search space
  num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 5)
  neurons_per_layer = trial.suggest_int("neurons_per_layer", 8, 128, step=8)
  epochs = trial.suggest_int("epochs", 10, 50, step=10)
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)
  batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
  optimizer_name = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'RMSprop'])
  weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

  # model init
  input_dim = 784
  output_dim = 10

  model = MYNN(input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate)
  model.to(device)

  # optimizer selection
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

  if optimizer_name == 'Adam':
    optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  elif optimizer_name == 'SGD':
    optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  else:
    optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

  # training loop

  for epoch in range(epochs):

    for batch_features, batch_labels in train_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

      # forward pass
      outputs = model(batch_features)

      # calculate loss
      loss = criterion(outputs, batch_labels)

      # back pass
      optimizer.zero_grad()
      loss.backward()

      # update grads
      optimizer.step()


  # evaluation
  model.eval()
  # evaluation on test data
  total = 0
  correct = 0

  with torch.no_grad():

    for batch_features, batch_labels in test_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

      outputs = model(batch_features)

      _, predicted = torch.max(outputs, 1)

      total = total + batch_labels.shape[0]

      correct = correct + (predicted == batch_labels).sum().item()

    accuracy = correct/total

  return accuracy



In [ ]:
Study= optuna.create_study(direction="maximize") #we have to maximize accuracy that is why maximize if we were required to minimize loss we would have used mionimize
Study.optimize(objective, n_trials=10)

[I 2026-01-17 06:19:30,151] A new study created in memory with name: no-name-58693440-f3eb-4874-a1f8-57ebaabf4b1c
[I 2026-01-17 06:20:55,504] Trial 0 finished with value: 0.8850833333333333 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 128, 'epochs': 20, 'learning_rate': 7.353538007276722e-05, 'dropout_rate': 0.30000000000000004, 'batch_size': 32, 'optimizer': 'Adam', 'weight_decay': 3.481211119497111e-05}. Best is trial 0 with value: 0.8850833333333333.
[I 2026-01-17 06:21:13,925] Trial 1 finished with value: 0.8573333333333333 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 56, 'epochs': 10, 'learning_rate': 0.0001457323806152521, 'dropout_rate': 0.5, 'batch_size': 128, 'optimizer': 'Adam', 'weight_decay': 0.00030321738707561526}. Best is trial 0 with value: 0.8850833333333333.
[I 2026-01-17 06:24:08,425] Trial 2 finished with value: 0.8305833333333333 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 8, 'epochs': 50, 'learning_rate': 0.01681

In [ ]:
Study.best_value

0.8941666666666667

In [ ]:
Study.best_params

{'num_hidden_layers': 4,
 'neurons_per_layer': 104,
 'epochs': 20,
 'learning_rate': 0.0006319644417674017,
 'dropout_rate': 0.1,
 'batch_size': 32,
 'optimizer': 'Adam',
 'weight_decay': 0.0006538507706263243}